# Overview
- [MLB lightGBM Starter Dataset&Code[en, ja]](https://www.kaggle.com/columbia2131/mlb-lightgbm-starter-dataset-code-en-ja/data)
- LightGBMによるbaseline

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from tqdm.notebook import tqdm
import lightgbm as lgb
from plotly import express as px
# import mlb

## Config

In [2]:
class CFG:
    INPUT_DIR = "../input/mlb-player-digital-engagement-forecasting"
    # INPUT_DIR = "../input/mlb-unnested-dataset" # for kaggle kernel

In [3]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date']

feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

## Load dataset

In [4]:
def load_dataset(input_path):
    INPUT_PATH = Path(input_path)
    train_next = pd.read_pickle(INPUT_PATH / "train_nextDayPlayerEngagement.pickle")
    train_rosters = pd.read_pickle(INPUT_PATH / "train_rosters.pickle")
    train_scores = pd.read_pickle(INPUT_PATH / "train_playerBoxScores.pickle")
    players = pd.read_pickle(INPUT_PATH / "players.pickle")

    return train_next, train_rosters, train_scores, players

In [5]:
train_next, train_rosters, train_scores, players = load_dataset(CFG.INPUT_DIR)

In [6]:
train_next.head()

,date,engagementMetricsDate,playerId,target1,target2,target3,target4
0,20180101,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,20180101,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,20180101,2018-01-02,519317,0.974327,56.177044,13.693746,64.166664
3,20180101,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,20180101,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373


In [7]:
train_rosters.head()

,date,playerId,gameDate,teamId,statusCode,status
0,20180101,400121,2018-01-01,116,A,Active
1,20180101,408045,2018-01-01,142,A,Active
2,20180101,425492,2018-01-01,120,A,Active
3,20180101,429664,2018-01-01,136,A,Active
4,20180101,431151,2018-01-01,121,A,Active


In [8]:
train_scores.head()

,date,home,gamePk,gameDate,gameTimeUTC,teamId,teamName,playerId,playerName,jerseyNum,...,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances
0,20180329,1,529418,2018-03-29,2018-03-29T23:08:00Z,119,Los Angeles Dodgers,605131,Austin Barnes,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20180329,1,529406,2018-03-29,2018-03-29T20:00:00Z,139,Tampa Bay Rays,605480,Mallex Smith,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,20180329,0,529416,2018-03-29,2018-03-29T20:10:00Z,143,Philadelphia Phillies,546318,Odubel Herrera,37,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,20180329,0,529412,2018-03-29,2018-03-29T20:05:00Z,108,Los Angeles Angels,527043,Jefry Marte,19,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
4,20180329,1,529408,2018-03-29,2018-03-29T20:15:00Z,118,Kansas City Royals,449181,Paulo Orlando,16,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0


In [9]:
# create dataset
train_scores = train_scores.groupby(["playerId", "date"]).sum().reset_index()
train = train_next[targets_cols].merge(players, on=["playerId"], how="left")
train = train.merge(train_rosters[rosters_cols], on=["playerId", "date"], how="left")
train = train.merge(train_scores[scores_cols], on=["playerId", "date"], how="left")

# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

## Training

In [10]:
train_X = train[feature_cols]
train_y = train[['target1', 'target2', 'target3', 'target4']]

_index = (train['date'] < 20210401)
x_train = train_X.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid = train_X.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)

In [11]:
def fit_lgb(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = lgb.LGBMRegressor(**params)
    model.fit(x_train, y_train, 
        eval_set=[(x_valid, y_valid)],  
        early_stopping_rounds=verbose, 
        verbose=verbose)
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score


# training lightgbm
params = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 100000,
 'learning_rate': 0.1,
 'random_state': 42,
}

oof1, model1, score1 = fit_lgb(
    x_train, y_train['target1'],
    x_valid, y_valid['target1'],
    params
)
oof2, model2, score2 = fit_lgb(
    x_train, y_train['target2'],
    x_valid, y_valid['target2'],
    params
)
oof3, model3, score3 = fit_lgb(
    x_train, y_train['target3'],
    x_valid, y_valid['target3'],
    params
)
oof4, model4, score4 = fit_lgb(
    x_train, y_train['target4'],
    x_valid, y_valid['target4'],
    params
)

score = (score1+score2+score3+score4) / 4
print(f'score: {score}')

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 0.736327
Early stopping, best iteration is:
[44]	valid_0's l1: 0.736321
mae: 0.7363212098026487
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 1.5341
[200]	valid_0's l1: 1.53482
Early stopping, best iteration is:
[117]	valid_0's l1: 1.52952
mae: 1.5295155668134564
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 0.566114
[200]	valid_0's l1: 0.566114
[300]	valid_0's l1: 0.566114
Early stopping, best iteration is:
[261]	valid_0's l1: 0.566114
mae: 0.5661144613680614
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 1.34221
[200]	valid_0's l1: 1.32964
[300]	valid_0's l1: 1.31703
[400]	valid_0's l1: 1.31154
[500]	valid_0's l1: 1.31154
Early stopping, best iteration is:
[401]	valid_0's l1: 1.31153
mae: 1.3115306688466417
score: 1.035870476707702


In [14]:
def visualize_feature_importance(models, feat_train_df) -> plotly.graph_objects.Figure:
    '''LightGBMのfeature importanceを可視化
    '''
    for target_i in range(len(models)):
        feature_importance_df = pd.DataFrame()
        for i, model in enumerate(models[target_i]):
            _df = pd.DataFrame()
            _df['feature_importance'] = model.feature_importance(importance_type="gain")
            _df['feature'] = feat_train_df.columns
            _df['model_no'] = i + 1
            feature_importance_df = pd.concat([feature_importance_df, _df], 
                                            axis=0, ignore_index=True)

        order = feature_importance_df.groupby('feature')\
            .mean()[['feature_importance']]\
            .sort_values('feature_importance', ascending=False).index[:50]
        
        fig = px.box(
            feature_importance_df.query("feature in @order"),
            x="feature_importance",
            y="feature",
            category_orders={"feature": order},
            width=1250,
            height=900,
            title=f"Target{target_i+1} Top 50 feature importance",
        )
        fig.update_yaxes(showgrid=True)
        fig.show()

NameError: name 'plotly' is not defined

## Inference

In [12]:
# players_cols = ['playerId', 'primaryPositionName']
# rosters_cols = ['playerId', 'teamId', 'status']
# scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
#        'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
#        'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
#        'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
#        'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
#        'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
#        'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
#        'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
#        'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
#        'groundOutsPitching', 'runsPitching', 'doublesPitching',
#        'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
#        'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
#        'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
#        'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
#        'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
#        'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
#        'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
#        'inheritedRunnersScored', 'catchersInterferencePitching',
#        'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
#        'assists', 'putOuts', 'errors', 'chances']

# null = np.nan
# true = True
# false = False

# env = mlb.make_env() # initialize the environment
# iter_test = env.iter_test() # iterator which loops over each date in test set

# for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
#     sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    
#     # creat dataset
#     sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
#                                         .map(lambda x: int(x.split('_')[1]))
#     # Dealing with missing values
#     if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
#         test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
#     else:
#         test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
#         for col in train_rosters.columns:
#             if col == 'playerId': continue
#             test_rosters[col] = np.nan
            
#     if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
#         test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
#     else:
#         test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
#         for col in train_scores.columns:
#             if col == 'playerId': continue
#             test_scores[col] = np.nan
#     test_scores = test_scores.groupby('playerId').sum().reset_index()
#     test = sample_prediction_df[['playerId']].copy()
#     test = test.merge(players[players_cols], on='playerId', how='left')
#     test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
#     test = test.merge(test_scores[scores_cols], on='playerId', how='left')

#     test['label_playerId'] = test['playerId'].map(player2num)
#     test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
#     test['label_teamId'] = test['teamId'].map(teamid2num)
#     test['label_status'] = test['status'].map(status2num)
    
#     test_X = test[feature_cols]
    
#     # predict
#     pred1 = model1.predict(test_X)
#     pred2 = model2.predict(test_X)
#     pred3 = model3.predict(test_X)
#     pred4 = model4.predict(test_X)
    
#     # merge submission
#     sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
#     sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
#     sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
#     sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
#     sample_prediction_df = sample_prediction_df.fillna(0.)
#     del sample_prediction_df['playerId']
    
#     env.predict(sample_prediction_df)